# Exploratory data analysis on London Data

This notebook will serve for exploring the London Dataset and gainging insight on how to process it 

First, we recuperate the necessary packages 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The data is divided in 163 datasets of 1 million entries.

We upload the data from several csv file and merge them together

In [15]:
df = pd.read_csv("../Data/Londondata/Power-Networks-LCL-June2015(withAcornGps)v2_150.csv")

Add as many datasets as needed

In [6]:
for i in range(2,20): 
    dataframe = pd.read_csv(r"../Data/Londondata/Power-Networks-LCL-June2015(withAcornGps)v2_"+str(i)+".csv")
    df = df.append(dataframe)

In [17]:
df.head()


,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
0,MAC002398,ToU,2013-04-10 21:30:00.0000000,0.192,ACORN-L,Adversity
1,MAC002398,ToU,2013-04-10 22:00:00.0000000,0.207,ACORN-L,Adversity
2,MAC002398,ToU,2013-04-10 22:30:00.0000000,0.268,ACORN-L,Adversity
3,MAC002398,ToU,2013-04-10 23:00:00.0000000,0.217,ACORN-L,Adversity
4,MAC002398,ToU,2013-04-10 23:30:00.0000000,0.29,ACORN-L,Adversity


The data ranges from 2012 to 2014, we will keep the data from 2013

In [54]:
#Get the year of observation
df["year"] = pd.to_datetime(df["DateTime"]).dt.year
df.index = df.index - 3840


#Isolate the year 2013
df = df[df["year"] == 2013]

Mak the usage be floats rather than strings

In [55]:
#Get rid of Null values
df["KWH/hh (per half hour) "] = df["KWH/hh (per half hour) "].replace('Null',np.nan)

#Get values as float
df["KWH/hh (per half hour) "] = pd.to_numeric(df["KWH/hh (per half hour) "], downcast="float")


Lets look at how many data points there is for each households (LCLid) for the 2013 year

In [56]:
df.groupby("LCLid")['KWH/hh (per half hour) '].apply(lambda x: len(x))



LCLid
MAC000036    35050
MAC000037    35050
MAC000038    35046
MAC000039    35034
MAC000040    35052
MAC000041    35048
MAC000042    35038
MAC000043    35048
MAC000045    35050
MAC000047    17518
MAC000049    35052
MAC000050     5092
MAC000053    35044
MAC000054    35050
MAC000055    35046
MAC000056    35048
MAC000057    35052
MAC000058    35048
MAC000059    35048
MAC000060    35052
MAC000061    35050
MAC000062    35046
MAC000063    12775
MAC000066    35048
MAC000067    35048
MAC000068    17520
MAC000069    35046
MAC000070    35048
MAC000072    35050
MAC000073    17517
             ...  
MAC000638    17532
MAC000639    17531
MAC000642    17530
MAC000643    17528
MAC000644    17532
MAC000645    17531
MAC000646    17523
MAC000647    17530
MAC000648    17530
MAC000649    17532
MAC000650    17531
MAC000651    17481
MAC000652    17532
MAC000654    17532
MAC000655    17514
MAC000656    17521
MAC000657    17528
MAC000658    17527
MAC000659    17524
MAC000660    17528
MAC000661    17530
MAC000

We can see that most Households have around 17530 entries, which makes sens as there are 17520 half hours in a year

Some are however far from this value. As we have plenty of data, lets delete all Household with less than 17520 data points


In [57]:
groups = df.groupby("LCLid")['KWH/hh (per half hour) '].apply(lambda x: len(x))
groups = groups[groups>17520]

ind = groups.index

df = df[df["LCLid"].isin(ind)]

In [58]:
len(ind)

405

In [59]:
grp = df.groupby("LCLid")['KWH/hh (per half hour) '].sum()

len(grp[grp == 0])

0

We now have N households (this is easily modulable by calling more datasets at the beginning) 

We however have more data points than half hours. Lets identify how. It can be some repeated half hour. Lets test for this on one household

In [60]:
house1 = df[df["LCLid"] == "MAC000036"]
house1[house1.duplicated()]
house1.head()

,Acorn,Acorn_grouped,DateTime,KWH/hh (per half hour),LCLid,stdorToU,year
-8946,ACORN-E,Affluent,2013-01-01 00:00:00.0000000,0.006,MAC000036,Std,2013
-8945,ACORN-E,Affluent,2013-01-01 00:30:00.0000000,0.017,MAC000036,Std,2013
-8944,ACORN-E,Affluent,2013-01-01 01:00:00.0000000,0.059,MAC000036,Std,2013
-8943,ACORN-E,Affluent,2013-01-01 01:30:00.0000000,0.000,MAC000036,Std,2013
-8942,ACORN-E,Affluent,2013-01-01 02:00:00.0000000,0.000,MAC000036,Std,2013


There is indeed some duplicates. Lets drop them

In [61]:
house1.drop_duplicates(inplace = True)
len(house1)

C:\Users\Kirsch\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


17519

We now have a normal number of values. There is just one value missing, which will be taken care of afterwards. First, lets modify the whole dataset

In [62]:
df.drop_duplicates(inplace = True)

print(len(df)/len(ind))

17517.723456790125


The average number of data points per households is now 17517, which is much better. It is however below the 17520 half hours. Lets find which half hours are missing and include them with a nan reading

In [63]:
df_full = df

In [68]:
year = pd.date_range("01/01/2013 00:00.0000000", "31/12/2013 23:30.0000000", freq="30min")

d = [None]*len(ind)
add = [None]*len(ind)
for i in range(len(i)):
    d[i] = df_full[df_full["LCLid"] == ind[i]]
    r = pd.to_datetime(d[i]["DateTime"])
    years = [x.strftime("%Y-%m-%d %H:%M:%S") for x in year]
    r = [x.strftime("%Y-%m-%d %H:%M:%S") for x in r]
    main_list = np.setdiff1d(list(years),list(r))

    add[i] = pd.DataFrame({"LCLid":ind[i], "DateTime": main_list,"KWH/hh (per half hour) ": np.nan, 'Acorn': "ACORN-A",
                          "Acorn_grouped": "Affluent", "year": 2013,'stdorToU': "Std"})

    df_full = df_full.append(add[i])
    print(i)
    
    
    
df_full = df_full.sort_values(['LCLid', 'DateTime'])

df_full = df_full.reset_index(drop = True)



0


In [84]:
8760*405

3547800

In [70]:
print(df_full.groupby("LCLid").size())

LCLid
MAC000036    17520
MAC000037    17520
MAC000038    17520
MAC000039    17520
MAC000040    17520
MAC000041    17520
MAC000042    17520
MAC000043    17520
MAC000045    17520
MAC000049    17520
MAC000053    17520
MAC000054    17520
MAC000055    17520
MAC000056    17520
MAC000057    17520
MAC000058    17520
MAC000059    17520
MAC000060    17520
MAC000061    17520
MAC000062    17520
MAC000066    17520
MAC000067    17520
MAC000069    17520
MAC000070    17520
MAC000072    17520
MAC000074    17520
MAC000077    17520
MAC000078    17520
MAC000081    17520
MAC000083    17520
             ...  
MAC000630    17520
MAC000631    17520
MAC000632    17520
MAC000634    17520
MAC000637    17520
MAC000638    17520
MAC000639    17520
MAC000642    17520
MAC000643    17520
MAC000644    17520
MAC000645    17520
MAC000646    17520
MAC000647    17520
MAC000648    17520
MAC000649    17520
MAC000650    17520
MAC000652    17520
MAC000654    17520
MAC000656    17520
MAC000657    17520
MAC000658    17520
MAC000

We now have the right number of data for each households. We however need to change the granularity of the data, for it to match the South African data 

In [71]:
df_full_h = df_full

In [72]:
df_full_h.index = pd.DatetimeIndex(df_full_h["DateTime"])
df_full_h = df_full_h.drop(["DateTime"], axis = 1)
df_full_h.head()

,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year
DateTime,,,,,,
2013-01-01 00:00:00,ACORN-E,Affluent,0.006,MAC000036,Std,2013
2013-01-01 00:30:00,ACORN-E,Affluent,0.017,MAC000036,Std,2013
2013-01-01 01:00:00,ACORN-E,Affluent,0.059,MAC000036,Std,2013
2013-01-01 01:30:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013
2013-01-01 02:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013


In [73]:
df_full_h[df_full_h['KWH/hh (per half hour) '].isnull()]
df_full_h["2013-03-26 21:00:00"]

,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year
DateTime,,,,,,
2013-03-26 21:00:00,ACORN-E,Affluent,0.044,MAC000036,Std,2013
2013-03-26 21:00:00,ACORN-E,Affluent,0.000,MAC000037,Std,2013
2013-03-26 21:00:00,ACORN-L,Adversity,0.395,MAC000038,Std,2013
2013-03-26 21:00:00,ACORN-F,Comfortable,0.562,MAC000039,Std,2013
2013-03-26 21:00:00,ACORN-A,Affluent,0.411,MAC000040,Std,2013
2013-03-26 21:00:00,ACORN-Q,Adversity,0.396,MAC000041,Std,2013
2013-03-26 21:00:00,ACORN-Q,Adversity,1.324,MAC000042,Std,2013
2013-03-26 21:00:00,ACORN-H,Comfortable,0.288,MAC000043,Std,2013
2013-03-26 21:00:00,ACORN-E,Affluent,0.652,MAC000045,Std,2013


In [75]:
type(df_full_h.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [2]:
df_full_h.to_csv('tempo_london.csv', sep = ',')

NameError: name 'df_full_h' is not defined

In [3]:
df_full_h = pd.read_csv('tempo_london.csv', sep = ',')

In [7]:
groups = df_full_h.groupby("LCLid")['KWH/hh (per half hour) '].apply(lambda x: len(x))

ind = groups.index

In [26]:
df_hour = df_full_h[df_full_h["LCLid"] == ind[0]]
df_hour.index = pd.to_datetime(df_hour["DateTime"])
print("eee")
df_hours = df_hour.resample('H').apply(lambda x: x.values.sum())
print("dddd")
df_hours["Acorn"] = df_hour["Acorn"]
df_hours["Acorn_grouped"] = df_hour["Acorn_grouped"]
df_hours["LCLid"] = ind[0]
df_hours['stdorToU'] = df_hour['stdorToU']
df_hours['year'] = df_hour['year']


for i in range(1, len(ind)):
    df_hour1 = df_full_h[df_full_h["LCLid"] == ind[i]]
    df_hour1.index = pd.to_datetime(df_hour1["DateTime"])
    df_hours1 = df_hour1.resample('H').apply(lambda x: x.values.sum())
    df_hours1["Acorn"] = df_hour1["Acorn"]
    df_hours1["Acorn_grouped"] = df_hour1["Acorn_grouped"]
    df_hours1["LCLid"] = ind[i]
    df_hours1['stdorToU'] = df_hour1['stdorToU']
    df_hours1['year'] = df_hour1['year']
    
    print(i)

    
    df_hours = df_hours.append(df_hours1)
    
df_hours = df_hours.sort_values(['LCLid','DateTime'])

eee
dddd
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

ValueError: 'DateTime' is both an index level and a column label, which is ambiguous.

We now have a dataset of the right number of entries for each household. 

Lets now study the distribution for each household, to see how it differs from one to another 

In [30]:
df_hours.drop("DateTime", axis = 1, inplace = True)

In [31]:
df_hours.head()

,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year
DateTime,,,,,,
2013-01-01 00:00:00,ACORN-E,Affluent,0.023,MAC000036,Std,2013
2013-01-01 01:00:00,ACORN-E,Affluent,0.059,MAC000036,Std,2013
2013-01-01 02:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013
2013-01-01 03:00:00,ACORN-E,Affluent,0.068,MAC000036,Std,2013
2013-01-01 04:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013
2013-01-01 05:00:00,ACORN-E,Affluent,0.069,MAC000036,Std,2013
2013-01-01 06:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013
2013-01-01 07:00:00,ACORN-E,Affluent,0.066,MAC000036,Std,2013
2013-01-01 08:00:00,ACORN-E,Affluent,0.013,MAC000036,Std,2013


In [32]:
df_hours.to_csv("../Data/Londondata/revised_new.csv")

len(df_hours)/8760

405.0

In [2]:
df_hours = pd.read_csv("../Data/Londondata/revised_new.csv")

## Having mean hourly data per weekday per season

### Getting all the info we need 

In [3]:
londonf = df_hours

In [10]:
londonf["Month"] = pd.to_datetime(londonf["DateTime"]).dt.month
londonf["Hour"] = pd.to_datetime(londonf["DateTime"]).dt.hour


In [11]:
londonf.tail()

,DateTime,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year,Month,Hour
3547795,2013-12-31 19:00:00,ACORN-L,Adversity,2.014,MAC000670,Std,2013,12,19
3547796,2013-12-31 20:00:00,ACORN-L,Adversity,0.411,MAC000670,Std,2013,12,20
3547797,2013-12-31 21:00:00,ACORN-L,Adversity,0.491,MAC000670,Std,2013,12,21
3547798,2013-12-31 22:00:00,ACORN-L,Adversity,0.400,MAC000670,Std,2013,12,22
3547799,2013-12-31 23:00:00,ACORN-L,Adversity,0.424,MAC000670,Std,2013,12,23


Careful, the seasons are opposite to the London seasons

In [12]:
season = londonf["Month"].where(
    (londonf["Month"] != 12), 'Winter').where(
    (londonf["Month"] != 1), 'Winter').where(
    (londonf["Month"] != 2), 'Winter').where(
    (londonf["Month"] != 3), 'Spring').where(
    (londonf["Month"] != 4), 'Spring').where(
    (londonf["Month"] != 5), 'Spring').where(
    (londonf["Month"] != 6), 'Summer').where(
    (londonf["Month"] != 7), 'Summer').where(
    (londonf["Month"] != 8), 'Summer').where(
    (londonf["Month"] != 9), 'Autumn').where(
    (londonf["Month"] != 10), 'Autumn').where(
    (londonf["Month"] != 11), 'Autumn')

In [13]:
season.unique()

array(['Winter', 'Spring', 'Summer', 'Autumn'], dtype=object)

In [14]:
londonf["Season"] = season

In [15]:
londonf.head()

,DateTime,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year,Month,Hour,Season
0,2013-01-01 00:00:00,ACORN-E,Affluent,0.023,MAC000036,Std,2013,1,0,Winter
1,2013-01-01 01:00:00,ACORN-E,Affluent,0.059,MAC000036,Std,2013,1,1,Winter
2,2013-01-01 02:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013,1,2,Winter
3,2013-01-01 03:00:00,ACORN-E,Affluent,0.068,MAC000036,Std,2013,1,3,Winter
4,2013-01-01 04:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013,1,4,Winter


In [16]:
londonf["Weekday"] = pd.to_datetime(londonf["DateTime"]).dt.weekday

In [19]:
londonf["Weekday"] = londonf["Weekday"].where(
    (londonf["Weekday"] != 0), 'Monday').where(
    (londonf["Weekday"] != 1), 'Tuesday').where(
    (londonf["Weekday"] != 2), 'Wednesday').where(
    (londonf["Weekday"] != 3), 'Thursday').where(
    (londonf["Weekday"] != 4), 'Friday').where(
    (londonf["Weekday"] != 5), 'Saturday').where(
    (londonf["Weekday"] != 6), 'Sunday')

In [20]:
londonf.head()

,DateTime,Acorn,Acorn_grouped,KWH/hh (per half hour),LCLid,stdorToU,year,Month,Hour,Season,Weekday
0,2013-01-01 00:00:00,ACORN-E,Affluent,0.023,MAC000036,Std,2013,1,0,Winter,Tuesday
1,2013-01-01 01:00:00,ACORN-E,Affluent,0.059,MAC000036,Std,2013,1,1,Winter,Tuesday
2,2013-01-01 02:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013,1,2,Winter,Tuesday
3,2013-01-01 03:00:00,ACORN-E,Affluent,0.068,MAC000036,Std,2013,1,3,Winter,Tuesday
4,2013-01-01 04:00:00,ACORN-E,Affluent,0.000,MAC000036,Std,2013,1,4,Winter,Tuesday


### Isolating the seasons and creating the mean profiles

In [53]:
summer = londonf[londonf["Season"] == "Summer"]
winter = londonf[londonf["Season"] == "Winter"]
autumn = londonf[londonf["Season"] == "Autumn"]
spring = londonf[londonf["Season"] == "Spring"]

##### Summer

In [54]:
summer_mean_gb = summer.groupby(["LCLid","Weekday","Hour"])['KWH/hh (per half hour) '].apply(lambda x: np.mean(x))

In [55]:
summer_mean_gb.head()

LCLid      Weekday  Hour
MAC000036  Friday   0       0.055308
                    1       0.040769
                    2       0.060308
                    3       0.048615
                    4       0.051615
Name: KWH/hh (per half hour) , dtype: float64

In [56]:
summer_mean = summer_mean_gb.to_frame()
summer_mean.head()

KWH/hh (per half hour) 
LCLid     Weekday Hour                         
MAC000036 Friday  0                    0.055308
                  1                    0.040769
                  2                    0.060308
                  3                    0.048615
                  4                    0.051615

In [57]:
summer_mean.reset_index(inplace = True)

In [58]:
summer_mean.head()

,LCLid,Weekday,Hour,KWH/hh (per half hour)
0,MAC000036,Friday,0,0.055308
1,MAC000036,Friday,1,0.040769
2,MAC000036,Friday,2,0.060308
3,MAC000036,Friday,3,0.048615
4,MAC000036,Friday,4,0.051615


In [59]:
summer_mean["season"] = "Summer"

summer_mean.columns = ['profile_ID', 'day_of_the_week', 'hour', 'consumption_kWh', 'season']

columns = ['profile_ID', 'season', 'day_of_the_week', 'hour', 'consumption_kWh']

summer_mean = summer_mean[columns]

summer_mean.head()

,profile_ID,season,day_of_the_week,hour,consumption_kWh
0,MAC000036,Summer,Friday,0,0.055308
1,MAC000036,Summer,Friday,1,0.040769
2,MAC000036,Summer,Friday,2,0.060308
3,MAC000036,Summer,Friday,3,0.048615
4,MAC000036,Summer,Friday,4,0.051615


##### Winter

In [60]:
winter_mean_gb = winter.groupby(["LCLid","Weekday","Hour"])['KWH/hh (per half hour) '].apply(lambda x: np.mean(x))

In [61]:
winter_mean_gb.head()

LCLid      Weekday  Hour
MAC000036  Friday   0       0.027833
                    1       0.032500
                    2       0.030583
                    3       0.023250
                    4       0.033000
Name: KWH/hh (per half hour) , dtype: float64

In [62]:
winter_mean = winter_mean_gb.to_frame()
winter_mean.reset_index(inplace = True)
winter_mean.head()

,LCLid,Weekday,Hour,KWH/hh (per half hour)
0,MAC000036,Friday,0,0.027833
1,MAC000036,Friday,1,0.032500
2,MAC000036,Friday,2,0.030583
3,MAC000036,Friday,3,0.023250
4,MAC000036,Friday,4,0.033000


In [63]:
winter_mean["season"] = "Winter"

winter_mean.columns = ['profile_ID', 'day_of_the_week', 'hour', 'consumption_kWh', 'season']

columns = ['profile_ID', 'season', 'day_of_the_week', 'hour', 'consumption_kWh']

winter_mean = winter_mean[columns]

winter_mean.head()

,profile_ID,season,day_of_the_week,hour,consumption_kWh
0,MAC000036,Winter,Friday,0,0.027833
1,MAC000036,Winter,Friday,1,0.032500
2,MAC000036,Winter,Friday,2,0.030583
3,MAC000036,Winter,Friday,3,0.023250
4,MAC000036,Winter,Friday,4,0.033000


##### Spring

In [64]:
spring_mean_gb = spring.groupby(["LCLid","Weekday","Hour"])['KWH/hh (per half hour) '].apply(lambda x: np.mean(x))

In [65]:
spring_mean_gb.head()

LCLid      Weekday  Hour
MAC000036  Friday   0       0.041714
                    1       0.025929
                    2       0.041071
                    3       0.022357
                    4       0.043143
Name: KWH/hh (per half hour) , dtype: float64

In [66]:
spring_mean = spring_mean_gb.to_frame()
spring_mean.reset_index(inplace = True)
spring_mean.head()

,LCLid,Weekday,Hour,KWH/hh (per half hour)
0,MAC000036,Friday,0,0.041714
1,MAC000036,Friday,1,0.025929
2,MAC000036,Friday,2,0.041071
3,MAC000036,Friday,3,0.022357
4,MAC000036,Friday,4,0.043143


In [67]:
spring_mean["season"] = "Spring"

spring_mean.columns = ['profile_ID', 'day_of_the_week', 'hour', 'consumption_kWh', 'season']

columns = ['profile_ID', 'season', 'day_of_the_week', 'hour', 'consumption_kWh']

spring_mean = spring_mean[columns]

spring_mean.head()

,profile_ID,season,day_of_the_week,hour,consumption_kWh
0,MAC000036,Spring,Friday,0,0.041714
1,MAC000036,Spring,Friday,1,0.025929
2,MAC000036,Spring,Friday,2,0.041071
3,MAC000036,Spring,Friday,3,0.022357
4,MAC000036,Spring,Friday,4,0.043143


##### Autumn

In [68]:
autumn_mean_gb = autumn.groupby(["LCLid","Weekday","Hour"])['KWH/hh (per half hour) '].apply(lambda x: np.mean(x))

In [69]:
autumn_mean_gb.head()

LCLid      Weekday  Hour
MAC000036  Friday   0       0.037077
                    1       0.035846
                    2       0.036231
                    3       0.033308
                    4       0.031769
Name: KWH/hh (per half hour) , dtype: float64

In [70]:
autumn_mean = autumn_mean_gb.to_frame()
autumn_mean.reset_index(inplace = True)
autumn_mean.head()

,LCLid,Weekday,Hour,KWH/hh (per half hour)
0,MAC000036,Friday,0,0.037077
1,MAC000036,Friday,1,0.035846
2,MAC000036,Friday,2,0.036231
3,MAC000036,Friday,3,0.033308
4,MAC000036,Friday,4,0.031769


In [71]:
autumn_mean["season"] = "Autumn"

autumn_mean.columns = ['profile_ID', 'day_of_the_week', 'hour', 'consumption_kWh', 'season']

columns = ['profile_ID', 'season', 'day_of_the_week', 'hour', 'consumption_kWh']

autumn_mean = autumn_mean[columns]

autumn_mean.head()

,profile_ID,season,day_of_the_week,hour,consumption_kWh
0,MAC000036,Autumn,Friday,0,0.037077
1,MAC000036,Autumn,Friday,1,0.035846
2,MAC000036,Autumn,Friday,2,0.036231
3,MAC000036,Autumn,Friday,3,0.033308
4,MAC000036,Autumn,Friday,4,0.031769


### Merging the data into one dataset

In [72]:
london_analysis = pd.concat([winter_mean, summer_mean, spring_mean, autumn_mean])

In [73]:
london_analysis.head()

,profile_ID,season,day_of_the_week,hour,consumption_kWh
0,MAC000036,Winter,Friday,0,0.027833
1,MAC000036,Winter,Friday,1,0.032500
2,MAC000036,Winter,Friday,2,0.030583
3,MAC000036,Winter,Friday,3,0.023250
4,MAC000036,Winter,Friday,4,0.033000


In [74]:
london_analysis.to_csv("../Data/Londondata/london_formatted_narrow.csv")